In [50]:
import jsonlines
import pandas as pd
import pickle
import pprint

from collections import Counter
from datasets import load_from_disk
from tqdm import tqdm

In [51]:
pp = pprint.PrettyPrinter(indent=2)
pd.set_option("display.max_rows", 500)

In [74]:
def get_domain(uri):
    return uri.split("/")[2]


def get_df(domains_count):
    domains = []
    counts = []
    for domain, count in domains_count.most_common():
        domains.append(domain)
        counts.append(count)

    total = sum(counts)
    ratio = [(100 * i) / total for i in counts]
    return pd.DataFrame.from_dict({"domain": domains, "count": counts, "ratio": ratio})

In [75]:
with open("/home/piktus_huggingface_co/1tt/data/wiki_domains.pkl", "rb") as handle:
    wiki_domains = pickle.load(handle)

In [97]:
wiki_df = get_df(wiki_domains)
wiki_df = wiki_df[:100]

In [77]:
with open("/home/piktus_huggingface_co/1tt/data/oscar_domains.pkl", "rb") as handle:
    oscar_domains = pickle.load(handle)

In [78]:
oscar_df = get_df(oscar_domains)
oscar_df[:8]

,domain,count,ratio
0,pubmed.ncbi.nlm.nih.gov,164199,0.038010
1,www.theguardian.com,102966,0.023835
2,unistore.www.microsoft.com,67582,0.015644
3,us.vestiairecollective.com,64876,0.015018
4,imgur.com,62244,0.014409
5,www.reuters.com,61665,0.014275
6,espas.secure.europarl.europa.eu,60999,0.014120
7,www.forbes.com,58727,0.013594


In [98]:
both = wiki_df.join(
    oscar_df.set_index("domain"),
    on="domain",
    how="left",
    lsuffix="_wiki",
    rsuffix="_oscar",
).fillna(0)

In [99]:
both

,domain,count_wiki,ratio_wiki,count_oscar,ratio_oscar
0,0,4321781,14.761861,0.0,0.000000e+00
1,books.google.com,923239,3.153498,11530.0,2.669027e-03
2,archive.org,422327,1.442538,2138.0,4.949158e-04
3,www.nytimes.com,286414,0.978301,2090.0,4.838045e-04
4,www.ncbi.nlm.nih.gov,282416,0.964645,1811.0,4.192201e-04
5,www.bbc.co.uk,222875,0.761272,6305.0,1.459516e-03
6,www.theguardian.com,192091,0.656123,102966.0,2.383513e-02
7,news.bbc.co.uk,166381,0.568306,41252.0,9.549236e-03
8,www.billboard.com,165065,0.563811,3915.0,9.062654e-04
9,www.census.gov,161907,0.553024,16765.0,3.880853e-03


In [100]:
MAX = 13131313
tot["ratio"] = tot.apply(
    lambda row: MAX if row["ratio_oscar"] == 0 else row["ratio_wiki"] / row["ratio_oscar"], axis=1
)

In [103]:
tot.sort_values(["ratio", "count_wiki"], ascending=False)

,domain,count_wiki,ratio_wiki,count_oscar,ratio_oscar,ratio
0,0,4321781,14.761861,0.0,0.000000e+00,1.313131e+07
11,news.google.com,127045,0.433946,0.0,0.000000e+00,1.313131e+07
22,tvbythenumbers.zap2it.com,58393,0.199452,0.0,0.000000e+00,1.313131e+07
26,www.stat.gov.pl,51732,0.176700,0.0,0.000000e+00,1.313131e+07
28,twitter.com,48608,0.166030,0.0,0.000000e+00,1.313131e+07
34,articles.latimes.com,43628,0.149020,0.0,0.000000e+00,1.313131e+07
42,www.facebook.com,37873,0.129362,0.0,0.000000e+00,1.313131e+07
50,int.soccerway.com,32654,0.111536,0.0,0.000000e+00,1.313131e+07
53,espn.go.com,31913,0.109005,0.0,0.000000e+00,1.313131e+07
54,www.huffingtonpost.com,30221,0.103226,0.0,0.000000e+00,1.313131e+07
